In [ ]:
#See README file to see where code is sourced from.

In [ ]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import keras
from keras.models import Sequential, Model,load_model
#from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D, Dropout
from keras.preprocessing import image
from keras.initializers import glorot_uniform
from tensorflow.keras.applications.resnet50 import preprocess_input
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
import scipy
import random
import gc
import glob
from keras.utils.vis_utils import plot_model



In [ ]:
#Read in CSV file for FER2013 data to be manipulated.
data = pd.read_csv('/com.docker.devenvironments.code/fer2013/icml_face_data.csv')

In [ ]:
#Define which CSV data is image data and which is label data.
pixel_data = data[' pixels']
label_data = data['emotion']

In [ ]:
#Print number of images that can be generated from CSV data. 
len(label_data)

In [ ]:
#Pre-process the CSV data and reshape it into a 48x48 image.
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    img = np.fromstring(pixel_data[i], dtype='int', sep=' ')
    img = img.reshape(48,48,1)
    images.append(img)

  X = np.array(images)
  return X



In [ ]:
#Duplicate data for emotion classes with lower samples, 'auto' means all will have the same number of examples.
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

In [ ]:
#Test that the reshaped data fits a 48x48 image.
X_over_series = pd.Series(X_over.flatten())
X_over_series

In [ ]:
#Preprocess data and turn into image form.
X = preprocess_pixels(X_over_series)
Y = Y_over

In [ ]:
Y = Y_over.values.reshape(Y.shape[0],1)
Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 45)
print(X_train.shape, Y_train.shape)

In [ ]:
#Test random image from dataset and display it to check that it has formed an image.
plt.imshow(X[25000,:,:,0])

In [ ]:
#Define new model.
def emotion_recognition(input_shape):

  X_input = Input(input_shape)

  X = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='same')(X_input)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = Conv2D(32, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.2)(X)

  X = Conv2D(64, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(64, (3,3), strides=(1,1), padding = 'same')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)


  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.3)(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)

  X = Conv2D(128, (3,3), strides=(1,1), padding = 'valid')(X)
  X = BatchNormalization(axis=3)(X)
  X = Activation('relu')(X)
  
  X = MaxPooling2D((2,2))(X)
  X = Dropout(0.4)(X)

  X = Flatten()(X)
  X = Dense(7, activation = 'softmax')(X)
  
  model = Model(inputs=X_input, outputs=X)
  

  return model

In [ ]:
#Load weights from training, define learning rate and compile model.
model = emotion_recognition((48,48,1))
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
#Summary of model's layers and parameters.
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
#Hot encode the emotion labels before training.
y_train = to_categorical(Y_train, num_classes=7)
y_test = to_categorical(Y_test, num_classes=7)


In [ ]:
#Define what each result from the prediction function means for each emotion.
label_dict = {0 : 'Angry', 1 : 'Disgust', 2 : 'Fear', 3 : 'Happiness', 4 : 'Sad', 5 : 'Surprise', 6 : 'Neutral'}


In [ ]:
#Prediction function, returns the vector of image label, as well as index of image location.
preds = model.predict(X_train)
def get_class(preds):
  pred_class = np.zeros((preds.shape[0],1))

  for i in range(len(preds)):
   pred_class[i] = np.argmax(preds[i])

  return pred_class

pred_class_train = get_class(preds)

In [ ]:
# DO NOT RUN AGAIN AS SAVED TRAINING CHECKPOINT IS LOADED
# Train model using csv data.
#checkpoint_path = "/com.docker.devenvironments.code/Checkpoints/cp.ckpt"
#checkpoint_dir = os.path.dirname(checkpoint_path)
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

#Model_Results = model.fit(X_train, y_train, epochs = 30, batch_size=64, validation_data=(X_test,y_test))
#model.save('Original Improved NO RESNET')


In [ ]:
#Load model to be tested.
model = load_model('Original Improved NO RESNET')
model.summary()

In [ ]:
#Pick a random image from the RandomImages folder, containing all test images from FER2013, not from CSV data.
test_image_count = len(os.listdir('/com.docker.devenvironments.code/fer2013/RandomImages'))
print('There are ' + str(test_image_count) + ' images in the testing folder')

test_image = random.choice(os.listdir('/com.docker.devenvironments.code/fer2013/RandomImages'))

#Load the selected image
img_path = ('/com.docker.devenvironments.code/fer2013/RandomImages/' + test_image)
img = image.load_img(img_path, grayscale=True, target_size=(48,48,1))

#Translate image into array form so it can be fed into the prediction function.
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

#Display the image and the predicted emotion as an output.

prediction = np.argmax(model.predict(x))
print('The predicted emotion is : ' + label_dict[prediction])
my_image = image.load_img(img_path)
plt.imshow(my_image)



In [ ]:
#Evaluate on Test Data:
results = model.evaluate(X_test, y_test, batch_size = 256)
print("test loss, test acc",results)

In [ ]:
#plt.subplot(2,1,1)
#plt.plot(Model_Results.history['accuracy'])
#plt.plot(Model_Results.history['val_accuracy'])
#plt.title('Original Model Improved - Accuracy')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc = 'lower right')

In [ ]:
#plt.subplot(2,1,2)
#plt.plot(Model_Results.history['loss'])
#plt.plot(Model_Results.history['val_loss'])
#plt.title('Original Model Improved- Loss')
#plt.ylabel('loss')
#plt.xlabel('epoch')
#plt.legend(['train', 'test'], loc = 'upper right')
